# Лабораторная работа 1

## Задание 1

### Описание

Реализовать клиентскую и серверную часть приложения. Клиент отсылает серверу
сообщение «Hello, server». Сообщение должно отразиться на стороне сервера.
Сервер в ответ отсылает клиенту сообщение «Hello, client». Сообщение должно
отобразиться у клиента.

- Обязательно использовать библиотеку `socket`
- Реализовать с помощью протокола UDP

Полезные ссылки:
-  https://habr.com/ru/post/149077/
-  https://andreymal.org/socket3/
-  https://docs.python.org/3.6/howto/sockets.html
-  https://docs.python.org/3.6/library/socket.html
-  https://www.youtube.com/watch?v=Lbfe3-v7yE0

### Код
#### Server

In [3]:
# %load "Task 1/server.py"
import socket


# UDP
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# Ensures that port is always ready to be used again
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind(('localhost', 12346))

# Makes keyboard interrupt possible at all times
sock.settimeout(1.0)

print(
    f"Started server at udp://{sock.getsockname()[0]}:{sock.getsockname()[1]}")

while True:
    try:
        connection, client_address = None, None
        try:
            connection, client_address = sock.recvfrom(2048)
        # Handle timeout
        except IOError:
            continue

        data = connection.decode('utf-8')
        print('Recived:', data)

        sock.sendto(b"Hello, client", client_address)

    except KeyboardInterrupt:
        print("Stopping server...")
        if connection:
            connection.close()
        break
sock.close()


Started server at tcp://127.0.0.1:12346
Stopping server...


#### Client
Так как Jupyter не хочет запускать cells в параллели, клиент запускается через командную строку:

python "./students/k33401/Reingeverts_Vadim/Lr1/Task 1/client.py"

In [5]:
# %load "Task 1/client.py"
import socket

# UDP
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.connect(('localhost', 12346))

sock.send(b"Hello, server")

try:
    connection = sock.recv(2048)
    data = connection.decode('utf-8')
    print('Recived:', data)
except ConnectionResetError:
    print("Could not connect to the server")

sock.close()


Could not connect to the server


## Задание 2

### Описание

Реализовать клиентскую и серверную часть приложения. Клиент запрашивает у
сервера выполнение математической операции, параметры, которые вводятся с
клавиатуры. Сервер обрабатывает полученные данные и возвращает результат
клиенту. Варианты:

1. **Теорема Пифагора**
2. Решение квадратного уравнения.
3. Поиск площади трапеции.
4. Поиск площади параллелограмма.

Вариант выбирается в соответствии с порядковым номером в журнале. Пятый
студент получает вариант 1 и т.д.

- Обязательно использовать библиотеку `socket`
- Реализовать с помощью протокола TCP

### Код
#### Server

In [19]:
# %load "Task 2/server.py"
import socket
from math import sqrt


def calc_pythagorean_equation(solveFor, x, y):
    solution = None
    x = float(x)
    y = float(y)
    
    if solveFor == "a":
        solution = sqrt(y**2 - x**2)
    elif solveFor == "b":
        solution = sqrt(y**2 - x**2)
    else:
        solution = sqrt(x**2 + y**2)
    return solution


# TCP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Ensures that port is always ready to be used again
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind(('localhost', 12346))


# Makes keyboard interrupt possible at all times
sock.settimeout(1.0)
sock.listen(10)

print(
    f"Started server at tcp://{sock.getsockname()[0]}:{sock.getsockname()[1]}")

while True:
    try:
        connection, client_address = None, None
        try:
            connection, client_address = sock.accept()
        # Handle timeout
        except IOError:
            continue
        data = connection.recv(2048)
        data = data.decode('utf-8')
        print('Recived:\n' + data)

        solveFor, x, y, _ = data.split("\n")
        solution = calc_pythagorean_equation(solveFor, x, y)

        print("Sending response:", solution)
        connection.send(str(solution).encode('utf-8'))

    except KeyboardInterrupt:
        print("Stopping server...")
        if connection:
            connection.close()
        break
sock.close()


Started server at tcp://127.0.0.1:12346
Recived:
a
2.0
2.0

Sending response: 0.0


#### Client
Так как Jupyter не хочет запускать cells в параллели, клиент запускается через командную строку:

python "./students/k33401/Reingeverts_Vadim/Lr1/Task 2/client.py"

In [12]:
# %load "Task 2/client.py"
import socket

# TCP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    sock.connect(('localhost', 12346))
    sock.send(b"Hello, server")

    connection = sock.recv(2048)
    data = connection.decode('utf-8')
    print('Recived:', data)
except ConnectionRefusedError:
    print("Could not connect to the server")


sock.close()

Could not connect to the server
